In [1]:
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2, 40).__str__()
import cv2
import torch
import torchvision.transforms as transforms
import numpy as np
import time

In [2]:
file1 = 'PWYL'
file2 = 'CS'
image = cv2.imread(f'{file1}/{file2}/{file1}-{file2}.jpg')
image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
data = torch.from_numpy(image)

(winW, winH) = (4096, 4096)
stepSize = (2048, 2048)
unfolded = data.unsqueeze(0).unfold(2, winW, stepSize[0]).unfold(3, winH, stepSize[1])
windows = unfolded.contiguous().view(-1, 3, winW, winH).detach()
# windows = windows.permute(0, 2, 1).view(num_windows, channels, winW, winH).numpy()
out_dir = f'dataset/{file1}/{file2}'
if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)

RuntimeError: maximum size for tensor at dimension 3 is 3 but size is 4096

In [ ]:
 # windows = windows.numpy()
s = time.perf_counter()
window = windows[0, :, :, :].detach()
to_pil = transforms.ToPILImage()
window = to_pil(window)
window.save(f'test.jpg')
e = time.perf_counter()
print(f'time2: {e - s}')
# torch.save(window, f'{out_dir}/{file1}_{file2}_{0}.pt')

In [2]:
def some_func(roi):
    '''
    simple function to return the mean of the region
    of interest
    '''
    return np.mean(roi)

In [7]:
img = np.zeros((37262,32763), dtype=np.uint8)
img.strides

(32763, 1)

In [18]:
def window_signal(signal, window, overlap):
    """ 
    Windowing function for data segmentation.

    Parameters:
    ------------
    signal: ndarray
            The signal to segment.
    window: int
            Window length, in samples.
    overlap: int
             Number of samples to overlap

    Returns: 
    --------
    nd-array 
            A copy of the signal array with shape (rows, window),
            where row = (N-window)//(window-overlap) + 1
    """
    N = signal.reshape(-1).shape[0]
    if (window == overlap):
        rows = N//window
        overlap = 0
    else:
        rows = (N-window)//(window-overlap) + 1
        miss = (N-window)%(window-overlap)
        if(miss != 0):
            print('Windowing led to the loss of ', miss, ' samples.')
    item_size = signal.dtype.itemsize 
    strides = (window - overlap) * item_size
    return np.lib.stride_tricks.as_strided(signal, shape=(rows, window),
                                           strides=(strides, item_size))

In [8]:
def Unfold(max_x, max_y, winH, winW, stepSize):
    x_points_l = np.array([i for i in range(0, max_x, stepSize[0]) if i + winH <= max_x])
    y_points_u = np.array([i for i in range(0, max_y, stepSize[1]) if i + winW <= max_y])

    x_wins_cnt = len(x_points_l)
    y_wins_cnt = len(y_points_u)
    wins_cnt = x_wins_cnt * y_wins_cnt

    w_lu = np.repeat([x_points_l], y_wins_cnt, axis=0)
    w_lu = np.repeat(w_lu, 2, axis=1)
    w_lu[:, 1::2] = np.repeat(y_points_u[:, np.newaxis], x_wins_cnt, axis=1)
    w_lu = w_lu.reshape(wins_cnt, 2)

    w_ru = np.copy(w_lu)
    w_ru[:, 0] += winW

    w_rd = np.copy(w_ru)
    w_rd[:, 1] += winH

    w_ld = np.copy(w_rd)
    w_ld[:, 0] -= winW

    windows = np.hstack((w_lu, w_ru, w_rd, w_ld))
    return windows


In [2]:
img = cv2.imread('PWYL/CS/PWYL-CS.jpg')

In [3]:
print(img.strides)

(111786, 3, 1)


In [10]:
max_x, max_y = img.shape[1], img.shape[0]
winH, winW = 4096, 4096
stepSize = (2048, 2048)

In [12]:
windows = Unfold(max_x, max_y, winH, winW, stepSize)
windows3 = np.zeros((238, 4096, 4096, 3))
for i in range(windows.shape[0]):
    windows3[i, :, :, :] = img[windows[i, 1]:windows[i, 5], windows[i, 0]:windows[i, 2]]

MemoryError: Unable to allocate 89.2 GiB for an array with shape (238, 4096, 4096, 3) and data type float64

In [31]:
windows1 = window_signal(img, 4096*4096, 2048*4096)

(32763, 37262, 3)
Windowing led to the loss of  5011630  samples.


In [13]:
# 获取图片形状
height, width, channels = img.shape

# 定义窗口大小和步幅
winH, winW = 4096, 4096
stepH, stepW = 2048, 2048

# 计算步幅
strides = (stepH * img.strides[0], stepW*img.strides[1], img.strides[0], img.strides[1], img.strides[2])

# 计算输出形状
out_shape = ((height - winH) // stepH + 1, (width - winW) // stepW + 1, winH, winW, channels)

# 使用 np.lib.stride_tricks.as_strided 生成滑动窗口
windows2 = np.lib.stride_tricks.as_strided(img, shape=out_shape, strides=strides)

windows2 = windows2.reshape(-1, winH, winW, channels)
print(windows2.shape)

(238, 4096, 4096, 3)


In [39]:
print(windows.shape)
print(windows2.shape)
print(windows2)

(238, 8)
(33168, 4096)
[[50 67 66 ...  1  0  0]
 [67 66 48 ...  0  0  1]
 [66 48 67 ...  0  1  0]
 ...
 [ 4  0  0 ...  1  0  0]
 [ 0  0  4 ...  0  0  1]
 [ 0  4  0 ...  0  1  0]]
